<a href="https://colab.research.google.com/github/YeomSuWoong/SOCAR-AI-BOOT-CAMP/blob/main/Assignment_1_ETA_%EC%98%88%EC%B8%A1_%EB%AA%A8%EB%8D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
from sklearn import ensemble
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import PredefinedSplit

/usr/local/lib/python3.7/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:17: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  "Since version 1.0, "


In [3]:
tada_eta = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/data/tada_eta.xlsx')
print(tada_eta.columns)
tada_eta.head()

Index(['ATA', 'id', 'api_eta', 'created_at_kst', 'month', 'pickup_lng',
       'pickup_lat', 'pickup_gu', 'driver_id', 'driver_lng', 'driver_lat',
       'hour'],
      dtype='object')


,ATA,id,api_eta,created_at_kst,month,pickup_lng,pickup_lat,pickup_gu,driver_id,driver_lng,driver_lat,hour
0,10.60,R4L9FL6NVEL4D0,5.65,2019-12-01T00:55:09,12,127.091399,37.624545,노원구,DST52944,127.069978,37.616472,0
1,9.20,R3A7JBY7CS6TPD,8.60,2019-12-01T00:09:39,12,127.023351,37.625564,강북구,DPC72995,127.026479,37.604711,0
2,10.85,R2BIUY1TDK14XE,10.53,2019-12-15T00:18:42,12,126.881154,37.481641,금천구,DPP94923,126.906225,37.452167,0
3,5.52,R31PU5GNHSGU6C,1.92,2019-07-31T00:24:11,7,126.988375,37.567897,중구,DCT73585,126.991170,37.566416,0
4,8.60,R41WR6N1VT42DJ,7.10,2019-07-05T00:40:34,7,126.988684,37.566716,중구,DPJ86237,127.006860,37.571164,0


In [4]:
enc = sklearn.preprocessing.OrdinalEncoder(dtype=np.int32)
ordinal = enc.fit_transform(np.array(tada_eta['pickup_gu']).reshape(-1,1))
tada_eta['pickup_gu'] = ordinal[:,0]
tada_eta['distance'] = ((tada_eta['pickup_lat']-tada_eta['driver_lat'])**2 + (tada_eta['pickup_lng']-tada_eta['driver_lng'])**2)*100000
tada_eta = tada_eta.drop(['id', 'created_at_kst', 'driver_id'],1)
tada_eta.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """


,ATA,api_eta,month,pickup_lng,pickup_lat,pickup_gu,driver_lng,driver_lat,hour,distance
0,10.60,5.65,12,127.091399,37.624545,8,127.069978,37.616472,0,52.400202
1,9.20,8.60,12,127.023351,37.625564,2,127.026479,37.604711,0,44.465889
2,10.85,10.53,12,126.881154,37.481641,7,126.906225,37.452167,0,149.731535
3,5.52,1.92,7,126.988375,37.567897,23,126.991170,37.566416,0,1.000535
4,8.60,7.10,7,126.988684,37.566716,23,127.006860,37.571164,0,35.014797


In [5]:
tada_eta = tada_eta.sample(frac=1, random_state=0).reset_index(drop=True)

In [6]:
# 위도 경도 데이터들만 따로 모아서 차원 축소 진행
tada_eta_red = tada_eta.drop(['ATA', 'api_eta', 'month', 'pickup_gu', 'hour', 'distance'], 1) # 전체 데이터에서 축소시킬 위도 경도 데이터만 뗴어놓음
tada_eta_red.head() 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


,pickup_lng,pickup_lat,driver_lng,driver_lat
0,126.913779,37.549010,126.913082,37.527922
1,127.089688,37.501710,127.084503,37.498104
2,126.972378,37.569360,126.972541,37.576297
3,126.974081,37.519973,127.001186,37.526196
4,127.025221,37.524640,127.026035,37.512465


In [8]:
# 위, 경도 데이터에 대한 차원 축소 진행
tada_eta_red_normalized = StandardScaler().fit(tada_eta_red).transform(tada_eta_red)
pca = PCA(n_components = 2, random_state = 0).fit(tada_eta_red_normalized)
tada_eta_red_pca = pca.transform(tada_eta_red_normalized)
print(tada_eta_red.shape, tada_eta_red_pca.shape)

(13916, 4) (13916, 2)


In [9]:
# 위, 경도를 제외한 피처 데이터
tada_eta_nred = np.array(tada_eta.drop(['ATA', 'pickup_lng', 'pickup_lat', 'driver_lng', 'driver_lat'], 1))
print(tada_eta_nred.shape)
print(tada_eta_nred)

(13916, 5)
[[ 8.32       12.         12.         22.         44.52197739]
 [ 3.05       12.         17.         11.          3.98906617]
 [ 5.          7.         22.         19.          4.81469879]
 ...
 [ 6.85       12.         19.         18.         65.93693906]
 [ 1.42        7.          0.         20.          0.55455545]
 [ 4.38       12.         14.          7.          3.75100634]]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [10]:
# 위, 경도를 제외한 피처 데이터에 차원 축소한 위,경도 데이터를 결합
new_feature = np.concatenate([tada_eta_red_pca, tada_eta_nred], 1)
new_label = np.array(tada_eta['ATA'])
print(new_feature.shape, new_label.shape)
#new_tada_eta = np.concatenate([new_label, new_feature], 1)

(13916, 7) (13916,)


In [12]:
#train = new_tada_eta[:12000]
#test = new_tada_eta[12000:]

In [13]:
# 과제에 주어진 대로 훈련피처와 훈련 라벨을 12000개로 두고 나머지를 테스트 데이터로 사용
x_train = new_feature[:12000]
y_train = new_label[:12000]
x_test = new_feature[12000:]
y_test = new_label[12000:]

In [14]:
X = np.concatenate((x_train,x_test), axis = 0)
y = np.concatenate((y_train, y_test), axis = 0)

In [15]:
pds = PredefinedSplit(test_fold=[-1]*len(x_train) + [0]*len(x_test))

In [16]:
param_grid = {'max_iter':[100, 200, 300],
             'max_depth': [2,3,4,5],
             'min_samples_leaf': [1, 20, 100],
             'learning_rate': [0.01, 0.02, 0.05],
             'loss': ['least_squares']}

In [17]:
grid_search = GridSearchCV(estimator = ensemble.HistGradientBoostingRegressor(random_state=0),
                          param_grid = param_grid,
                          cv = pds, n_jobs = -1, scoring = 'neg_mean_squared_error', verbose = 2)

In [18]:
grid_search.fit(X,y)
print(grid_search.best_params_)

Fitting 1 folds for each of 108 candidates, totalling 108 fits


/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_hist_gradient_boosting/gradient_boosting.py:1306: FutureWarning: The loss 'least_squares' was deprecated in v1.0 and will be removed in version 1.2. Use 'squared_error' which is equivalent.
  FutureWarning,


{'learning_rate': 0.05, 'loss': 'least_squares', 'max_depth': 3, 'max_iter': 200, 'min_samples_leaf': 100}


In [19]:
params = {'max_iter':200,
         'max_depth':3,
         'min_samples_leaf':100,
         'learning_rate':0.05,
         'loss':'least_squares'}

In [20]:
reg = ensemble.HistGradientBoostingRegressor(**params, random_state=0)
reg.fit(x_train, y_train)

mse = mean_squared_error(y_test, reg.predict(x_test))
print("The mean squared error (MSE) on test set: {:.4f}".format(mse))

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_hist_gradient_boosting/gradient_boosting.py:1306: FutureWarning: The loss 'least_squares' was deprecated in v1.0 and will be removed in version 1.2. Use 'squared_error' which is equivalent.
  FutureWarning,


The mean squared error (MSE) on test set: 8.1813
